In [12]:
import itertools
import numpy as np
import random
import re
import os
#from os import path
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
from datetime import datetime
import time

import torch
from transformers import AutoModel, AutoTokenizer
import torch, torch.nn as nn
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import json

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [40]:
#### Parameters ####

basepath = "../data/raw/"

bert_type = 'roberta-base'
batch_size = 128

output_folder = "../outputs/"

test_label_folder = "../outputs/modifiers_span_predictions_roberta-base_2022-07-13_01_48_39_2022-07-13_01_57_02/"

n_epochs = 10

tasks = ['mp', 'me', 'qual']

train_doc_path = basepath + "train/text/"
trial_doc_path = basepath + "trial/txt/"
eval_doc_path = basepath + "eval/text/"

train_label_path = basepath + "train/tsv/"
trial_label_path = basepath + "trial/tsv/"

In [15]:
def extract_modifiers(data_filepath: str, output_filepath: str, quant_filepath: str):

    # Extracted keywords and units for faster processing   
    list_units = ['varves','hour','s−1','g mol−1','mg/l','kgs-1','beach materials','kW','g/m3','scale heights','μs','×','cm−1','mm2','mM'
            'Mbps','mbar','hr','passages','participants','μbar','day','m2 g−1','mg','oN','μg/m','kg','GPa','μM','women'
            'stems/ha','nbar','vertices','U/ml','m thick','month','ppt','W m−2','centimeters','second','mg/ml','mWm−2','°C','μg/L','wt.%'
            'times','months','ppq','M','km','mBar','cm2/Vs','Ma','H','point','monomer units','mA/cm2','nT','cm3 g−1','m'
            '°','metre','°N','bp','v/v','Mt','cores','Whitehall II participants','Rs','m/s','mdeg','Torr','Å/s','occasions','clones'
            'men','%','kHz','MPa','mH','MW','orders of magnitude','year-old','m s−1','UT','hours','h','μL','weeks','discrete particles'
            '% per year','wt%','cm−3','kV','fold','lines of code','days','nJ','years','km/h','week','km s−1','ppm','mmol/L','year'
            'Rp','elderly participants','g cm−3','bit','AU','MeV','mm per side','° latitude','mm3','yrs','pairs per mm','K/min',
            'Mg ha−1 year−1','wt. %','K''men','%','kHz','MPa','mH','MW','orders of magnitude','year-old','m s−1','UT','hours','h','μL','weeks','discrete particles'
            'byr','kR','s','mA g− 1','°S','min','mbsl','m−2','°C/min','W/m2','minutes','percentage points per year','nm','vol%','degrees'
            'mg cm− 2','pH','keV','mA','ka','employees','fold per passage','mm','RRh','eV','ms','μg','kg s−1','w/w','item'
            'μg/ml','μm','p0','g/L','horizons','V','Hz','SDG vertices','cm','items','percent','mg/L','ppm by mass','‰','g'
            'Saturn radii RS'
        ]

    set_units = set(list_units)
    print(set_units)

    # IsApproximate IsCount IsRange IsList IsMean IsMedian IsMeanHasSD IsMeanHasTolerance IsRangeHasTolerance HasTolerance                 

    revlist = sorted(list(set_units), reverse = True, key = len) # List of units
    # print(revlist)

    # we set non alphabet as the word boundary in our regex
    listcounts = ['[^a-z]half[^a-z]', '[^a-z]quarter[^a-z]', '[^a-z]one[^a-z]', '[^a-z]two[^a-z]', '[^a-z]three[^a-z]', '[^a-z]four[^a-z]', '[^a-z]five[^a-z]', '[^a-z]six[^a-z]', '[^a-z]seven[^a-z]', '[^a-z]eight[^a-z]', '[^a-z]nine[^a-z]', '[^a-z]ten[^a-z]', 
                  '[^a-z]eleven[^a-z]', '[^a-z]twelve[^a-z]', '[^a-z]thirteen[^a-z]','[^a-z]fourteen[^a-z]','[^a-z]fifteen[^a-z]','[^a-z]sixteen[^a-z]','[^a-z]seventeen[^a-z]','[^a-z]eighteen[^a-z]','[^a-z]nineteen[^a-z]','[^a-z]twenty[^a-z]',
                  '[^a-z]thirty[^a-z]','[^a-z]forty[^a-z]','[^a-z]fifty[^a-z]','[^a-z]sixty[^a-z]','[^a-z]seventy[^a-z]','[^a-z]eighty[^a-z]','[^a-z]ninety[^a-z]','[^a-z]hundred[^a-z]',
                  '[^a-z]thousand[^a-z]','[^a-z]million[^a-z]','[^a-z]billion[^a-z]','[^a-z]trillion[^a-z]']

    listcounts2 = []
    listcounts3 = []
    listcounts4 = []
    for item in listcounts:
        temp = "^"+item[6:]
        listcounts2.append(temp)

    # print(listcounts2)

    for item in listcounts:
        temp = item[0:len(item)-6] + "$"
        listcounts3.append(temp)

    # print(listcounts3)

    for item in listcounts:
        temp = "^"+item[6:len(item)-6]+"$"
        listcounts4.append(temp)

    # print(listcounts4)

    listcounts.extend(listcounts2)
    listcounts.extend(listcounts3)
    listcounts.extend(listcounts4)
    # print(listcounts)

    listapproximate = ['∼','~', 'about', 'around', 'close to', 'the order of','approximately', 'nominally', 'near', 'roughly', 'almost', 'approximation',
    '≈', 'circa']

    listmean = ['average', 'mean']

    listmedian = ['median']

    listrange = ['to', 'from', 'below', 'beyond', 'above', 'between', 'up to', '<', '>', 'upper',
    'greater', 'lesser', 'bigger', 'smaller', 'more than', 'less than', '≥', '≤', 'within',
    'throughout', 'at least', 'or more', 'or less', 'past', 'higher', 'almost', 'high', 'before', 'after',
    'over', 'under', 'range', 'ranging', 'top', 'at most', 'down to', '⩽', '⩾', '≳', 'as much as','±']

    listhyphen = ['−', '-']

    listlist = ['or', 'and']



    '''
    We set any non alphabet Character as the word boundary.
    Since scientific document can have non-unicode
    ''' 


    def findmodifier(sent, start_off, end_off):
        Numpresent = False
        IsUnit = False
        IsApproximate = False
        IsCount = True
        IsRange = False
        IsList = False 
        Mean = False
        IsMean = False
        IsMedian = False
        Tolerance = False
        Range = False
        IsMeanHasSD = False
        IsMeanHasTolerance = False
        IsRangeHasTolerance = False
        HasTolerance = False
        Unit = ""

        # We find if plus-minus exists


        t_sent = sent[start_off:end_off]
        print("Span is: ", t_sent)

        if re.search("[^a-z][0-9]|^[0-9]", t_sent.lower()) is not None:
            Numpresent = True
        else:
            for item in listcounts:
                if re.search(item, t_sent.lower()) is not None:
                    Numpresent = True

        if Numpresent is False:
            IsCount = False
            return ["null", IsApproximate, IsCount, IsRange, IsList, IsMean, IsMedian, IsMeanHasSD, IsMeanHasTolerance, IsRangeHasTolerance, HasTolerance]

        for unit in revlist:
            if re.search("[^a-zA-Z]"+unit+"[^a-zA-Z]"+'|'+"[^a-zA-Z]"+unit+"$", sent[start_off:end_off]) is not None:
                Unit+=unit
                IsUnit = True
                IsCount = False
                break

        if '±' in t_sent:
            IsCount = False
            Tolerance = True

        for item in listmean:
            tmp = re.search("[^a-z]"+item+"[^a-z]"+'|'+"[^a-z]"+item+"$" +'|'+"^"+item+"[^a-z]", sent[start_off:end_off].lower())
            if tmp is not None:
                Mean = True


        if re.search("[^a-z]"+"sd"+"[^a-z]"+"|"+"[^a-z]"+"sd"+"$", sent[start_off-20:end_off].lower()) is True:
            IsMeanHasSD = True

        for item in listrange:
            tmp = re.search("[^a-z]"+item+"[^a-z]"+'|'+"[^a-z]"+item+"$" +'|'+"^"+item+"[^a-z]", t_sent.lower())# sent[max(0,start_off-10):end_off].lower())
            if tmp is not None:
                Range = True

        tmp = re.search("[−|-]", t_sent)
        if tmp is not None:
            tmp1 = re.search("[0-9]+", sent[max(0,tmp.start()+start_off-10):tmp.start()+start_off])
            if tmp1 is not None:
                Range = True

        if re.search('[^a-z]'+'median'+'[^a-z]'+'|'+'^'+'median'+'[^a-z]'+'|'+'[^a-z]'+'median'+'$', t_sent.lower()):
            IsMedian = True

        tmp = re.search("[^a-z]and[^a-z]|[^a-z]or[^a-z]", t_sent.lower())
        if tmp is not None:
            tmp1 = re.search("[0-9]+", sent[start_off+tmp.end():end_off])
            tmp2 = re.search("[0-9]+", sent[max(0,start_off-1):(tmp.start()+start_off)])
            if tmp1 is not None and tmp2 is not None:
                if Range is not True:
                    IsList = True
                IsCount = False


        for item in listapproximate:
            tmp = re.search("[^a-z]"+item+"[^a-z]"+'|'+"[^a-z]"+item+"$" +'|'+"^"+item+"[^a-z]", sent[max(0,start_off-30):end_off].lower())
            if tmp is not None:
                IsApproximate = True

        if Range and not Tolerance:
            IsRange = True
        if IsMean and Tolerance and not IsMeanHasSD:
            IsMeanHasTolerance = True
        if Tolerance and Range:
            IsRangeHasTolerance = True
        if Tolerance and not IsMeanHasSD and not IsMeanHasTolerance and not IsRangeHasTolerance:
            HasTolerance = True

        if IsUnit is True:
            return [Unit, IsApproximate, IsCount, IsRange, IsList, IsMean, IsMedian, IsMeanHasSD, IsMeanHasTolerance, IsRangeHasTolerance, HasTolerance]
        else:
            return ["null", IsApproximate, IsCount, IsRange, IsList, IsMean, IsMedian, IsMeanHasSD, IsMeanHasTolerance, IsRangeHasTolerance, HasTolerance]


    def convert_output_to_dict(preds):
        label_dict = {}
        if preds[0] != "null":
            label_dict['unit'] = preds[0]

        mods = []

        mods_order_ = ["IsApproximate", "IsCount", "IsRange", "IsList", "IsMean", "IsMedian",
                        "IsMeanHasSD", "IsMeanHasTolerance", "IsRangeHasTolerance","HasTolerance"
                    ]

        for i, bool_value in enumerate(preds):
            if i == 0:
                pass
            else:
                if bool_value:
                    mods.append(mods_order_[i-1])

        if len(mods) > 0:
            label_dict["mods"] = mods

        if len(label_dict) == 0:
            return ""

        return str(label_dict)

    import json

    def label_and_dump(text_path, quant_path, save_path, trial_or_train=False):
        all_text = {f[:-4]: open(text_path + f, encoding="utf-8", errors='replace').read() for f in os.listdir(text_path)}


        if trial_or_train:
            trim_tsv = lambda x: x[:-4] if x[-4:] == ".tsv" else x
            quant_labels = {trim_tsv(k):v for k, v in json.load(open(quant_path)).items()}
        else:
            quant_labels = json.load(open(quant_path))

        '''
        Quant_labels:
        Dict{
                Doc1_id: {sent1_offset: [list of quant offsets for sent1],
                        sent2_offset: [list of quant offsets for sent2],
                            ....
                        }
                Doc2_id ....
                ....
        }
        Output:
        Dict{
                Doc1_id: [[docId, annotSet, annotType, startOffset, endOffset, annotId, text, other, sentence_start, sentence_end]
                            repeat same for second quant identified
                            ....
                        ]
                Doc2_id ....
                ....
        }
        '''
        assert set(sorted(all_text.keys())) == set(sorted(quant_labels.keys())), (quant_labels.keys(), "   |||||    ", all_text.keys())

        task2_output = {}
        for docid in quant_labels.keys():
            doc_txt = all_text[docid]
            ann_id = 1
            this_doc_labels = []
            this_sents_offs = sorted([int(x) for x in list(quant_labels[docid].keys())])
            for i, sent_offset in enumerate(this_sents_offs):
                this_sentence_start = sent_offset
                this_sentence_end = this_sents_offs[i+1] if (i +1) < len(this_sents_offs) else len(doc_txt)
                sent = doc_txt[this_sentence_start:this_sentence_end]

                sent_offset_str = str(sent_offset)
                single_sent_quant_offs = quant_labels[docid][sent_offset_str]

                for single_quant_offs in single_sent_quant_offs:
                    others_val = convert_output_to_dict(findmodifier(sent, single_quant_offs[0], single_quant_offs[1]))

                    this_offset = (single_quant_offs[0] + sent_offset, single_quant_offs[1] + sent_offset)
                    this_doc_labels.append([docid, ann_id, "Quantity",
                                            this_offset[0], this_offset[1], ann_id,
                                            doc_txt[this_offset[0]:this_offset[1]],
                                            others_val, this_sentence_start, this_sentence_end
                                            ])
                    ann_id += 1

            task2_output[docid] = this_doc_labels

        json.dump(task2_output, open(save_path, 'w+'), indent=2)

    train_doc_path = data_filepath + "train/text/"
    trial_doc_path = data_filepath + "trial/txt/"
    test_doc_path = data_filepath + "eval/text/"
    folder_name = output_filepath + bert_type.replace('/', '_') + '_' + datetime.today().strftime('%Y-%m-%d_%H_%M_%S')
    if os.path.isdir(folder_name):
        os.system("rm -rf " + folder_name)
    os.mkdir(folder_name)

    print("\n\n\n====================================================")
    print("====================== Train =======================")
    print("====================================================\n\n\n")


    label_and_dump(train_doc_path, quant_filepath + "train_spans.json",
                    os.path.join(folder_name, "train_labels.json"), True
                )

    print("\n\n\n====================================================")
    print("======================= Trial ======================")
    print("====================================================\n\n\n")

    label_and_dump(trial_doc_path, quant_filepath + "trial_spans.json",
                    os.path.join(folder_name, "trial_labels.json"), True
                )

    print("\n\n\n====================================================")
    print("======================= Test =======================")
    print("====================================================\n\n\n")

    label_and_dump(test_doc_path, quant_filepath + "test_spans.json",
                    os.path.join(folder_name + "/test_labels.json"), False
                )

In [9]:
extract_modifiers(data_filepath="../data/raw/", output_filepath= "../outputs/modifiers_", quant_filepath="../outputs/span_predictions_roberta-base_2022-07-13_01_48_39/")

{'weeks', 'kgs-1', 'W/m2', 'min', 'womenstems/ha', 'mbar', 'μm', 'nT', 'mBar', 'mg', 'h', 'kg', 'hours', 'bit', '°C/min', 'yearRp', 'cm', 'cm2/Vs', 'SDG vertices', '°S', 'μbar', 'cm−1', 'ppt', 'mH', 'nm', 'cores', 'μL', 'pH', 'metre', 'mg/l', 'hr', 'MPa', 'kg s−1', 'km/h', 'mdeg', 'p0', 'ppq', 'H', 'm/s', 'mm3', 'lines of code', 'mm', 'ka', 's−1', 'clonesmen', 'mm per side', 'μM', 'Mg ha−1 year−1', 'M', 'beach materials', 'mA', 'Torr', 'ppm', 'μg/m', '° latitude', 'second', 'RRh', 'mm2', 'cm−3', 'Rs', 'year-old', 'yrs', 'wt.%times', 'minutes', 'g mol−1', 'm s−1', 'ms', 'MW', 'orders of magnitude', 'mbsl', 'kHz', 'participants', 'kV', 'μg', 'm thick', 'days', 'V', 'mWm−2', 'Whitehall II participants', '°N', 'vol%', 'items', 'μg/L', 'km s−1', 'horizons', 'elderly participants', 'mg/L', 'mg/ml', 'itemμg/ml', 'kW', 's', 'percentage points per year', 'U/ml', 'gSaturn radii RS', 'Mt', 'employees', 'month', 'centimeters', 'percent', 'K/min', 'Hz', 'nJ', 'kR', 'keV', 'w/w', 'UT', 'years', 'mMM

Span is:  40 Hz
Span is:  2.30
Span is:  95%
Span is:  1.03–5.13),
Span is:  8.5 mm
Span is:  1.5 mm
Span is:  1.5 mm
Span is:  0.5 Hz
Span is:  1.8 cm
Span is:  greater than 70%.
Span is:  1.5 mm
Span is:  8.5 mm
Span is:  8.5 mm
Span is:  8.5 mm
Span is:  50 μm);
Span is:  200 μm).
Span is:  51-bp
Span is:  about 40%
Span is:  9 and 21
Span is:  6 weeks
Span is:  2.5 years
Span is:  115%
Span is:  169%
Span is:  0.001)
Span is:  50 μm
Span is:  7200 K
Span is:  12.4%
Span is:  0.13%
Span is:  10 cm
Span is:  20–40 cm
Span is:  0–10 cm
Span is:  1.5418 Å
Span is:  5° to 85°,
Span is:  0.1°
Span is:  4 s
Span is:  0.9).
Span is:  0.64%
Span is:  26 October 2011,
Span is:  0.02%
Span is:  December 16
Span is:  48 hr
Span is:  10%
Span is:  about −100 °C
Span is:  147 patients
Span is:  before 6 months
Span is:  before 9 months
Span is:  8.5 mm
Span is:  1180 and 1980 mm
Span is:  4 weeks



======================= Trial ======================



Span is:  2005–2010
Span is:  14 times
Sp

In [16]:
torch.manual_seed(434)

In [29]:
#### Train one epoch ####
def train(model, dataset, criterion, epoch_num):
    # Put the model to train
    model.train()

    # Lets keep track of the losses at each update
    train_losses = []
    num_batch = 0

    loss_scaling = {'me': 1.0, 'mp': 1.0, 'qual': 1.0}
    print("Loss Scale:", loss_scaling)
    
    for batch in dataset.batched_dataset:
        # Unpack the batch
        (texts, labels, att_masks, _, _, _, _) = batch

        preds = model(texts, att_masks, labels['QUANT'])

        # Take into account padded while calculating loss
        loss = 0
        for single_task, single_task_preds in preds.items():
            loss_unreduced = criterion(single_task_preds.permute(0,2,1), labels[single_task.upper()])
            loss += ((loss_unreduced * att_masks).sum() / (att_masks).sum()) * loss_scaling[single_task]
        loss.backward()

        # Update model weights
        optimizer.step()
        optimizer.zero_grad()

        if num_batch % 10 == 0:
            print("Train loss at {}:".format(num_batch), loss.item())

        num_batch += 1
        train_losses.append(loss.item())

    return np.average(train_losses)

def evaluate(model, tasks, dataset, criterion, epoch_num):
    # Put the model to eval mode
    model.eval()

    # Keep track of predictions
    valid_losses = []
    predicts, gnd_truths = {}, {}
    if 'me' in tasks:
        predicts['me'] = []
        gnd_truths['me'] = []
    if 'mp' in tasks:
        predicts['mp'] = []
        gnd_truths['mp'] = []
    if 'qual' in tasks:
        predicts['qual'] = []
        gnd_truths['qual'] = []

    loss_scaling = {'me': 1.0, 'mp': 1.0, 'qual': 1.0}
    print("Loss Scale:", loss_scaling)

    with torch.no_grad():
        for batch in dataset.batched_dataset:
            # Unpack the batch
            (texts, labels, att_masks, _, _, _, _) = batch
            # Make predictions on the model
            preds = model(texts, att_masks, labels['QUANT'])


            # Take into account padded while calculating loss
            loss = 0
            for single_task, single_task_preds in preds.items():
                loss_unreduced = criterion(single_task_preds.permute(0,2,1), labels[single_task.upper()])
                loss += ((loss_unreduced * att_masks).sum() / (att_masks).sum()) * loss_scaling[single_task]

            assert preds.keys() == predicts.keys()
            assert preds.keys() == gnd_truths.keys()
            
            # Get argmax of non-padded tokens
            for task in preds.keys():
                for sent_preds, sent_labels, sent_att_masks in zip(preds[task], labels[task.upper()], att_masks):
                    for token_preds, token_labels, token_masks in zip(sent_preds, sent_labels, sent_att_masks):
                        if token_masks.item() != 0:
                            predicts[task].append(token_preds.argmax().item())
                            gnd_truths[task].append((token_labels.item()))
            valid_losses.append(loss.item())

            assert len(predicts) == len(gnd_truths)

    # Create confusion matrix and evaluate on the predictions
    for task in preds.keys():
        print("\nTask:", task)
        target_names = ["NOT_" + task.upper(), task.upper()]
        
        confuse_mat = confusion_matrix(gnd_truths[task], predicts[task])
        classify_report = classification_report(gnd_truths[task], predicts[task],
                                        target_names=target_names,
                                        output_dict=True)

        print(confuse_mat)

        for labl in target_names:
            print(labl, "F1-score:", classify_report[labl]["f1-score"])
        print("Accu:", classify_report["accuracy"])
        print("F1-Weighted", classify_report["weighted avg"]["f1-score"])
        print("F1-Avg", classify_report["macro avg"]["f1-score"])

    mean_valid_loss = np.average(valid_losses)
    print("\nValidation loss", mean_valid_loss)

    return mean_valid_loss

In [30]:
class METestDataset:
    def __init__(self, text_path, quant_label_path):
        self.text_path = text_path
        self.quant_label_path = quant_label_path

        # Load all text
        self.textset = self.get_doc_ids()

        # Load all annotations
        all_files_with_text = list(self.textset.keys())
        all_quant_labels = json.load(open(self.quant_label_path))
        sorted(list(all_quant_labels.keys())) == sorted(all_files_with_text)
        print(len(all_files_with_text), len(all_quant_labels))
        
        # Also preprocess - normalizing numbers
        self.all_data = self.load_dataset(all_files_with_text, all_quant_labels)
        print("Loaded and processed data")

        # Load Tokenizer and Batch
        self.bert_tok = AutoTokenizer.from_pretrained(bert_type, use_fast=True)
        print("Loaded tokenizer")
        # Add new tokens in tokenizer
        new_special_tokens_dict = {"additional_special_tokens": ["<E>", "</E>"]}
        self.bert_tok.add_special_tokens(new_special_tokens_dict)

        self.batched_dataset = self.batch_dataset(self.all_data)

    def load_dataset(self, all_files_with_text, all_quant_labels):

        alldata = []
        normalize = lambda x: re.sub(r'\d', '0', x)

        # Load annotations from files with label
        for file_id in all_files_with_text:
            doc_text = self.textset[file_id]

            for quants in all_quant_labels[file_id]:
                this_quant_sent = doc_text[quants[8]:quants[9]]
                this_quant_offsets_from_sent_start = [quants[3] - quants[8], quants[4] - quants[8]]
                alldata.append([normalize(this_quant_sent), quants, this_quant_offsets_from_sent_start])

        return alldata

    def get_doc_ids(self):
        textset = {}
        for fn in os.listdir(self.text_path):
            with open(self.text_path+fn, encoding='utf-8', errors='replace') as textfile:
                text = textfile.read()
                textset[fn[:-4]] = text

        return textset


    def batch_dataset(self, sentence_mapped_data):
        """
            Inputs:
                    [ [sent, list of quant1_details from task2, quant_offset],
                      [sent, list of quant2_details from task2],
                      .....
                    ]
            Outputs:
                Batched data: [doc_ixs, sent_offsets, sentences, token_offsets, pad_masks]
        """
        flattened = sentence_mapped_data
        print(f'Flattened doc - {len(sentence_mapped_data)}',
                "\nSome examples:", flattened[:2]
            )

        cls_token_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize(self.bert_tok.cls_token))[0]
        sep_token_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize(self.bert_tok.sep_token))[0]
        pad_token_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize(self.bert_tok.pad_token))[0]

        special_st_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize("<E>"))
        special_end_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize("</E>"))
        assert len(special_st_idx) == 1
        assert len(special_end_idx) == 1
        special_st_idx, special_end_idx = special_st_idx[0], special_end_idx[0]

        print("CLS, SEP, PAD, <E>, </E> tokens are:", cls_token_idx, sep_token_idx, pad_token_idx, special_st_idx, special_end_idx)

        dataset = []
        idx = 0
        num_data = len(flattened)

        while idx < num_data:
            batch_quant_data = []
            batch_raw_text = []
            batch_raw_labels = []

            for single_sentence, quant_data, quant_offsets_from_sent_start in \
                        flattened[idx:min(batch_size, num_data)]:
                offset = quant_offsets_from_sent_start
                single_raw_anns = [[offset[0], offset[1], 'QUANT']]

                single_raw_text = single_sentence

                batch_quant_data.append(quant_data)
                batch_raw_text.append(single_raw_text)
                batch_raw_labels.append(single_raw_anns)

            batched_dict = self.bert_tok.batch_encode_plus(batch_raw_text,
                                return_offsets_mapping=True, padding=True)


            # Create sequence labels using token offsets
            no_special_batch_tokens = batched_dict['input_ids']
            no_special_batch_pads = batched_dict['attention_mask']
            no_special_batch_offset_mapping = batched_dict['offset_mapping']
            
            batch_tokens = []
            batch_pads = []
            batch_offset_mapping = []
            batch_labels = {"QUANT": []}

            # Add <E> and </E>
            for single_sent_tokens, single_sent_pad_masks, single_token_offs, single_sent_anns in \
                    zip(no_special_batch_tokens, no_special_batch_pads, no_special_batch_offset_mapping, batch_raw_labels):

                this_tokens = []
                this_pads = []
                this_offset_mapping = []
                this_labels = {"QUANT": []}

                quant_label, entity_label, property_label, qual_label = None, None, None, None
                for ann in single_sent_anns:
                    if ann[2] == "QUANT":
                        quant_label = [ann[0], ann[1]]
                    else:
                        raise ann

                quant_happening = 0
                for i, off in enumerate(single_token_offs):
                    if off == (0,0):
                        if quant_happening == 1: 
                            # Append for </E>, if the code reaches here then quant label has ended
                            this_tokens.append(special_end_idx)
                            this_pads.append(1)
                            this_offset_mapping.append((0,0))
                            this_labels["QUANT"].append(1)
                        quant_happening = 0

                        this_tokens.append(single_sent_tokens[i])
                        this_pads.append(single_sent_pad_masks[i])
                        this_offset_mapping.append(off)
                        this_labels["QUANT"].append(0)
                    else:
                        if off[1] < quant_label[0] or off[0] > quant_label[1]:
                            if quant_happening == 1: 
                                # Append for </E>, if the code reaches here then quant label has ended
                                this_tokens.append(special_end_idx)
                                this_pads.append(1)
                                this_offset_mapping.append((0,0))
                                this_labels["QUANT"].append(1)
                            quant_happening = 0
                        else:
                            if quant_happening == 0:
                                # Append for <E>, if the code reaches here then quant label has started
                                this_tokens.append(special_st_idx)
                                this_pads.append(1)
                                this_offset_mapping.append((0,0))
                                this_labels["QUANT"].append(1)
                            quant_happening = 1

                        # Check for all_quant and append
                        this_tokens.append(single_sent_tokens[i])
                        this_pads.append(single_sent_pad_masks[i])
                        this_offset_mapping.append(off)

                        if off[1] >= quant_label[0] and off[0] <= quant_label[1]:
                            this_labels["QUANT"].append(1)
                        else:
                            this_labels["QUANT"].append(0)

                batch_tokens.append(this_tokens)
                batch_pads.append(this_pads)
                batch_offset_mapping.append(this_offset_mapping)
                batch_labels["QUANT"].append(this_labels["QUANT"])

            max_batch_len = max([len(bt) for bt in batch_tokens])
            min_batch_len = min([len(bt) for bt in batch_tokens])
            assert max_batch_len == min_batch_len or max_batch_len == min_batch_len + 2

            token_pad_function_maxlen = lambda x, maxlen: x if len(x) == maxlen else x + [pad_token_idx, pad_token_idx]
            mask_pad_function_maxlen = lambda x, maxlen: x if len(x) == maxlen else x + [0, 0]
            label_pad_function_maxlen = lambda x, maxlen: x if len(x) == maxlen else x + [0, 0]

            batch_tokens = [token_pad_function_maxlen(sentence, max_batch_len) for sentence in batch_tokens]
            batch_tokens = torch.LongTensor(batch_tokens).to(device)

            batch_pads = [mask_pad_function_maxlen(sentence, max_batch_len) for sentence in batch_pads]
            pad_masks = torch.LongTensor(batch_pads).to(device)

            batch_maxlen = batch_tokens.shape[-1]
            for k in batch_labels:
                padded_labels = [label_pad_function_maxlen(sentence, max_batch_len) for sentence in batch_labels[k]]
                batch_labels[k] = padded_labels
            batch_ann_labels = {k: torch.LongTensor(batch_labels[k]).to(device)
                                    for k in batch_labels
                                }

            b = batch_size if (idx + batch_size) < num_data else (num_data - idx)
            assert batch_tokens.size() == torch.Size([b, batch_maxlen])
            for vvvvv in batch_ann_labels.values():
                assert vvvvv.size() == torch.Size([b, batch_maxlen])
            assert pad_masks.size() == torch.Size([b, batch_maxlen])

            dataset.append((batch_tokens, batch_ann_labels, pad_masks, batch_offset_mapping, batch_quant_data))
            idx += batch_size

        print("num_batches=", len(dataset), " | num_data=", num_data)
        return dataset

In [31]:
typemap = {"Quantity": "QUANT",
           "MeasuredEntity": "ME", 
           "MeasuredProperty": "MP", 
           "Qualifier": "QUAL"
        }

# Dataloader
class MEDataset:
    def __init__(self, text_path, label_path, bert_type):
        self.text_path = text_path
        self.label_path = label_path

        # Load all text
        self.textset = self.get_doc_ids()

        # Load all annotations
        files_with_label = [file_name[:-4] for file_name in os.listdir(label_path)]
        all_files_with_or_without_label = [file_name[:-4] for file_name in os.listdir(text_path)]
        files_without_label = list(set(all_files_with_or_without_label).difference(set(files_with_label)))
        print(len(files_with_label), len(files_without_label), len(all_files_with_or_without_label))
        self.all_data = self.load_dataset(files_with_label, files_without_label)

        # Map ME, MP, Qual to the Quant.
        self.mapped_data = self.map_dataset(self.all_data)

        # Preprocess - sentence splitting, normalizing numbers
        self.tokenized_data = self.tokenize_split_data(self.all_data)
        self.all_sentence_mapped_data = self.sentence_map(self.tokenized_data, self.mapped_data)
        print("Loaded and processed data")

        # Load Tokenizer and Batch
        self.bert_tok = AutoTokenizer.from_pretrained(bert_type)
        print("Loaded tokenizer")
        # Add new tokens in tokenizer
        new_special_tokens_dict = {"additional_special_tokens": ["<E>", "</E>"]}
        self.bert_tok.add_special_tokens(new_special_tokens_dict)
        print("LEN VOCAB:", len(self.bert_tok))
        self.batched_dataset = self.batch_dataset(self.all_sentence_mapped_data,
                                                  shuffle=True if "train" in text_path else False
                                                )

    def load_dataset(self, files_with_label, files_without_label):

        alldata = []

        # Load annotations from files with label
        for fn_no_ext in files_with_label:
            fn = fn_no_ext + ".tsv"
            entities = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
            with open(self.label_path+fn, encoding='utf-8', errors='replace') as annotfile:
                text = self.textset[fn[:-4]]
                next(annotfile)
                annots = annotfile.read().splitlines()
                for a in annots:
                    annot = a.split("\t")
                    atype = typemap[annot[2]]
                    start = int(annot[3])
                    stop = int(annot[4])
                    other = None if annot[-1] == '' else json.loads(annot[-1])
                    ann_id = annot[1]
                    # This is where we toss out the overlaps:
                    overlap = False
                    # for ent in entities[atype]:
                    #     if ((start >= ent[0] and start <= ent[1]) or (stop >= ent[0] and stop <= ent[1]) or
                    #         (ent[0] >= start and ent[0] <= stop) or (ent[1] >= start and ent[1] <= stop)):
                    #         overlap = True
                    if overlap == False:    
                        entities[atype].append((start, stop, atype, ann_id, other))
                alldata.append((text,
                                {"QUANT": entities["QUANT"],
                                 "ME": entities["ME"],
                                 "MP": entities["MP"],
                                 "QUAL": entities["QUAL"]
                                },
                                (fn,)
                            ))

        return alldata

    def get_doc_ids(self):
        textset = {}
        for fn in os.listdir(self.text_path):
            with open(self.text_path+fn, encoding='utf-8', errors='replace') as textfile:
                text = textfile.read()
                textset[fn[:-4]] = text

        return textset

    def map_dataset(self, all_data):
        '''
        Output:
            [
                [text,
                 List[
                        q1_start, q1_stop,
                        q1_atype, q1_ann_id,
                        [list of max length 3, containing corresponding ME, MP, QUAL]
                 ],
                 (filename,
                 )
                ]
            ]
        '''
        mapped_dataset = []
        for doc in all_data:
            this_quant_data = []
            for single_quant in doc[1]['QUANT']:
                this_ann_id = single_quant[3]
                this_quant_props = []
                for me in doc[1]['ME']:
                    if this_ann_id == me[-2]:
                        this_quant_props.append(me)
                for mp in doc[1]['MP']:
                    if this_ann_id == mp[-2]:
                        this_quant_props.append(mp)
                for qual in doc[1]['QUAL']:
                    if this_ann_id == qual[-2]:
                        this_quant_props.append(qual)

                this_quant_data.append([single_quant[0], single_quant[1],
                                        single_quant[2], single_quant[3],
                                        single_quant[4], this_quant_props
                                    ])

            mapped_dataset.append((doc[0], this_quant_data, doc[-1]))
        return mapped_dataset

    def tokenize_split_data(self, all_data):
        processed_data = []

        cnt_toks = {"figs.": 0, "fig.": 0, "et al.": 0,
                    "ref.": 0, "eq.": 0, "e.g.": 0,
                    "i.e.": 0, "nos.": 0, "no.": 0,
                    "spp.": 0
                    }
        regex_end_checker = [".*[a-zA-Z]figs\.$", 
                            ".*[a-zA-Z]fig\.$",
                            ".*[a-zA-Z]et al\.$",
                            ".*[a-zA-Z]ref\.$",
                            ".*[a-zA-Z]eq\.$",
                            ".*[a-zA-Z]e\.g\.$",
                            ".*[a-zA-Z]i\.e\.$",
                            ".*[a-zA-Z]nos\.$",
                            ".*[a-zA-Z]no\.$",
                            ".*[a-zA-Z]spp\.$",
                            # figs., fig., et al., Ref., Eq., e.g., i.e., Nos., No., spp.
                        ]

        assert len(cnt_toks) == len(regex_end_checker)

        # list of sentences
        # for every tokenized sentence obtained
            # check if ends with "fig or Figs or et al."
            # keep track of the index where the this sentence starts and end,

        all_tokenized_data = []
        for doc in all_data:
            flag = False
            sentences = sent_tokenize(doc[0])

            fixed_sentence_tokens = []
            curr_len = 0
            for s in sentences:
                if flag == True:
                    assert s[0] != ' '
                    white_length = doc[0][curr_len:].find(s[0])

                    prev_len = len(fixed_sentence_tokens[-1])
                    fixed_sentence_tokens[-1] = fixed_sentence_tokens[-1] + (" "*white_length) + s

                    assert fixed_sentence_tokens[-1][prev_len+white_length] == doc[0][curr_len+white_length], (fixed_sentence_tokens[-1], doc[0], curr_len, tmp_this_sent_len)
                    tmp_this_sent_len = white_length + len(s)
                    assert fixed_sentence_tokens[-1][-1] == doc[0][curr_len+tmp_this_sent_len-1], (fixed_sentence_tokens[-1], doc[0], curr_len, tmp_this_sent_len)
                    curr_len += tmp_this_sent_len
                else:
                    if len(fixed_sentence_tokens) != 0:
                        assert s[0] != ' '
                        white_length = doc[0][curr_len:].find(s[0])
                        fixed_sentence_tokens.append( (" "*white_length) + s )
                    else:
                        fixed_sentence_tokens.append(s)
                    assert fixed_sentence_tokens[-1][0] == doc[0][curr_len], (fixed_sentence_tokens, doc[0], curr_len, tmp_this_sent_len)
                    tmp_this_sent_len = len(fixed_sentence_tokens[-1])
                    assert fixed_sentence_tokens[-1][-1] == doc[0][curr_len+tmp_this_sent_len-1], (fixed_sentence_tokens[-1], doc[0], curr_len, tmp_this_sent_len)
                    curr_len += tmp_this_sent_len

                lower_cased_s = fixed_sentence_tokens[-1].lower()
                flag = False
                for i, k in enumerate(cnt_toks):
                    this_regex_pattern = regex_end_checker[i]
                    if lower_cased_s.endswith(k) and re.match(this_regex_pattern, lower_cased_s) == None:
                        cnt_toks[k] += 1
                        flag = True
                        break

            all_tokenized_data.append(fixed_sentence_tokens)      
        print("Fixed sentence splitting:", cnt_toks)
        return all_tokenized_data

    def sentence_map(self, all_tokenized_data, mapped_data):
        """
            Inputs:
                all_tokenized_data: list of [list of sentences]
                mapped_data: list of Tuple[doc, quant-wise-annotations, (doc_id,)]
            Outputs:
                all_annotated_split_data: 
                        [
                            Dict{'doc_id': doc_id,
                                'sentences': [list of string]
                                'offsets': [list of int]
                                'annotations': [ list of [sent_ix, sent_offset, offset_adjusted annotations for a single quant]
                                                 ... same for other quants
                                                ],
                    
                                ... Repeat for the second doc
                ]
        """

        # in the next loop
            # Replace all numbers by zero.
            # check if any of the annotations are used for this falls in between the start and end. Make sure no overlap
            # add offset as well.

        normalize = lambda x: re.sub(r'\d', '0', x)
        all_annotated_split_data = []
        
        for doc, sent_splits in zip(mapped_data, all_tokenized_data):

            this_offsets = []

            prev_end = 0
            for s in sent_splits:
                this_offsets.append([prev_end, prev_end+len(s)])
                prev_end += len(s)
            
            this_annotations = [] # Each element of the format: (sent_ix, anns)

            for ann in doc[1]:
                if len(ann[-1]) == 0:
                    lowest_char_ix = ann[0]
                    highest_char_ix = ann[1]
                else:
                    lowest_char_ix = min(ann[0], min(rel_ann[0] for rel_ann in ann[-1]))
                    highest_char_ix = max(ann[1], max(rel_ann[1] for rel_ann in ann[-1]))
                
                iii = 0
                # Calc low ix.
                min_sent_ix = -1
                while iii < len(this_offsets):
                    if lowest_char_ix < this_offsets[iii][1]:
                        min_sent_ix = iii
                        break
                    iii += 1

                # Calc high sent ix
                max_sent_ix = -1
                while iii < len(this_offsets):
                    if highest_char_ix <= this_offsets[iii][1]:
                        max_sent_ix = iii
                        break
                    iii += 1
                assert min_sent_ix != -1
                assert max_sent_ix != -1

                if min_sent_ix != max_sent_ix:
                    # skip, since it will be difficult during test time.
                    pass
                else:
                    # Append ann
                    this_ann_off = this_offsets[min_sent_ix][0]
                    offset_modified_ann = [ann[0]-this_ann_off, ann[1]-this_ann_off, ann[2], ann[3], ann[4],
                                            [(rel_ann[0]-this_ann_off, rel_ann[1]-this_ann_off, rel_ann[2], rel_ann[3], rel_ann[4])
                                                for rel_ann in ann[5]
                                            ]
                                        ]
                    this_annotations.append([min_sent_ix, this_ann_off, offset_modified_ann])

            all_annotated_split_data.append({'doc_id': doc[-1][0],
                        'sentences': [normalize(ss) for ss in sent_splits],
                        'offsets': this_offsets,
                        'annotations': this_annotations
                    })
            # print(all_annotated_split_data[-1])
            for ta in this_annotations:
                assert ta[0] >= 0 and ta[0] < len(sent_splits)
            assert len(all_annotated_split_data[-1]['offsets']) == len(all_annotated_split_data[-1]['sentences'])
            # assert len(all_annotated_split_data[-1]['offsets']) == len(all_annotated_split_data[-1]['annotations'])
        
        return all_annotated_split_data

    def batch_dataset(self, sentence_mapped_data, shuffle=False, batch_size=batch_size, device=device):
        """
            Inputs:
                sentence_mapped_data: 
                        [
                            Dict{'doc_id': doc_id,
                                'sentences': [list of string]
                                'offsets': [list of int]
                                'annotations': [ list of [sent_ix, sent_offset, offset_adjusted annotations for a single quant]
                                                 ... same for other quants
                                                ],
                    
                                ... Repeat for the second doc
                ]
            Outputs:
                Batched data: [doc_ixs, sent_offsets, sentences, labels, pad_masks]
        """
        # First flatten and remove overlapping labels and then shuffle
        # Then Batch
        flattened = [[doc['doc_id'], doc['sentences'][doc['annotations'][i][0]],
                      doc['annotations'][i][1], doc['annotations'][i]]
                    for doc in sentence_mapped_data for i in range(len(doc['annotations']))
                ]

        print(f'Flattened {len(sentence_mapped_data)} docs into {len(flattened)} data points',
                "\nSome examples:", flattened[:2])

        def has_overlap(ann, datapt):
            no_overlap = True
            sss = sorted([ann[:2]] + [list(ff[:2]) for ff in ann[-1]], key = lambda x: x[0])
            for i in range(len(sss)-1):
                if sss[i][1] > sss[i+1][0]:
                    no_overlap = False
                    print("\n\nWARNING: Discarding due to overlap", datapt, '\n\n')
                    break
            return no_overlap

        flattened = [f for f in flattened if has_overlap(f[-1][-1], f)]
        print("Length after discarding overlaps:", len(flattened))
        
        if shuffle:
            random.shuffle(flattened)

        cls_token_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize(self.bert_tok.cls_token))[0]
        sep_token_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize(self.bert_tok.sep_token))[0]
        pad_token_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize(self.bert_tok.pad_token))[0]
        
        special_st_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize("<E>"))
        special_end_idx = self.bert_tok.convert_tokens_to_ids(self.bert_tok.tokenize("</E>"))
        assert len(special_st_idx) == 1
        assert len(special_end_idx) == 1
        special_st_idx, special_end_idx = special_st_idx[0], special_end_idx[0]

        print("CLS, SEP, PAD, <E>, </E> tokens are:", cls_token_idx, sep_token_idx, pad_token_idx, special_st_idx, special_end_idx)

        dataset = []
        idx = 0
        num_data = len(flattened)
        while idx < num_data:
            batch_doc_ids = []
            batch_sent_offsets = []
            batch_raw_text = []
            batch_raw_labels = []

            for single_docid, single_sentence, single_offset, single_annotations_with_offset_sentence_idx in \
                        flattened[idx:min(idx+batch_size, num_data)]:
                
                single_annotations = single_annotations_with_offset_sentence_idx[-1]
                single_raw_anns = [single_annotations[:3]] + [list(ff[:3]) for ff in single_annotations[-1]]

                # quant_st, quant_end = (single_raw_anns[0][0], single_raw_anns[0][1])
                single_raw_text = single_sentence#[:quant_st] + " <E> " + \
                            #single_sentence[quant_st:quant_end] + " </E> " + single_sentence[quant_end:]
                

                batch_doc_ids.append(single_docid)
                batch_sent_offsets.append(single_offset)
                batch_raw_text.append(single_raw_text)
                batch_raw_labels.append(single_raw_anns)

            batched_dict = self.bert_tok.batch_encode_plus(batch_raw_text,
                                                        return_offsets_mapping=True,
                                                        padding=True)


            # Create sequence labels using token offsets
            no_special_batch_tokens = batched_dict['input_ids']
            no_special_batch_pads = batched_dict['attention_mask']
            no_special_batch_offset_mapping = batched_dict['offset_mapping']
            
            batch_tokens = []
            batch_pads = []
            batch_offset_mapping = []
            batch_labels = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}

            # Add <E> and </E>
            for single_sent_tokens, single_sent_pad_masks, single_token_offs, single_sent_anns in \
                    zip(no_special_batch_tokens, no_special_batch_pads, no_special_batch_offset_mapping, batch_raw_labels):

                this_tokens = []
                this_pads = []
                this_offset_mapping = []
                this_labels = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}

                quant_label, entity_label, property_label, qual_label = None, None, None, None
                for ann in single_sent_anns:
                    if ann[2] == "QUANT":
                        quant_label = [ann[0], ann[1]]
                    elif ann[2] == "ME":
                        entity_label = [ann[0], ann[1]]
                    elif ann[2] == "MP":
                        property_label = [ann[0], ann[1]]
                    elif ann[2] == "QUAL":
                        qual_label = [ann[0], ann[1]]
                    else:
                        raise ann

                quant_happening = 0
                for i, off in enumerate(single_token_offs):
                    if off == (0,0):
                        if quant_happening == 1: 
                            # Append for </E>, if the code reaches here then quant label has ended
                            this_tokens.append(special_end_idx)
                            this_pads.append(1)
                            this_offset_mapping.append((0,0))
                            this_labels["QUANT"].append(1)
                            this_labels["MP"].append(0)
                            this_labels["ME"].append(0)
                            this_labels["QUAL"].append(0)
                        quant_happening = 0

                        this_tokens.append(single_sent_tokens[i])
                        this_pads.append(single_sent_pad_masks[i])
                        this_offset_mapping.append(off)
                        this_labels["QUANT"].append(0)
                        this_labels["MP"].append(0)
                        this_labels["ME"].append(0)
                        this_labels["QUAL"].append(0)
                    else:
                        if off[1] < quant_label[0] or off[0] > quant_label[1]:
                            if quant_happening == 1: 
                                # Append for </E>, if the code reaches here then quant label has ended
                                this_tokens.append(special_end_idx)
                                this_pads.append(1)
                                this_offset_mapping.append((0,0))
                                this_labels["QUANT"].append(1)
                                this_labels["MP"].append(0)
                                this_labels["ME"].append(0)
                                this_labels["QUAL"].append(0)
                            quant_happening = 0
                        else:
                            if quant_happening == 0:
                                # Append for <E>, if the code reaches here then quant label has started
                                this_tokens.append(special_st_idx)
                                this_pads.append(1)
                                this_offset_mapping.append((0,0))
                                this_labels["QUANT"].append(1)
                                this_labels["MP"].append(0)
                                this_labels["ME"].append(0)
                                this_labels["QUAL"].append(0)
                            quant_happening = 1

                        # Check for all_quant and append
                        this_tokens.append(single_sent_tokens[i])
                        this_pads.append(single_sent_pad_masks[i])
                        this_offset_mapping.append(off)

                        if off[1] >= quant_label[0] and off[0] <= quant_label[1]:
                            this_labels["QUANT"].append(1)
                        else:
                            this_labels["QUANT"].append(0)

                        if entity_label != None and off[1] >= entity_label[0] and off[0] <= entity_label[1]:
                            this_labels["ME"].append(1)
                        else:
                            this_labels["ME"].append(0)

                        if property_label != None and off[1] >= property_label[0] and off[0] <= property_label[1]:
                            this_labels["MP"].append(1)
                        else:
                            this_labels["MP"].append(0)

                        if qual_label != None and off[1] >= qual_label[0] and off[0] <= qual_label[1]:
                            this_labels["QUAL"].append(1)
                        else:
                            this_labels["QUAL"].append(0)

                batch_tokens.append(this_tokens)
                batch_pads.append(this_pads)
                batch_offset_mapping.append(this_offset_mapping)
                batch_labels["QUANT"].append(this_labels["QUANT"])
                batch_labels["MP"].append(this_labels["MP"])
                batch_labels["ME"].append(this_labels["ME"])
                batch_labels["QUAL"].append(this_labels["QUAL"])

            batch_tokens = torch.LongTensor(batch_tokens).to(device)
            pad_masks = torch.LongTensor(batch_pads).to(device)

            batch_maxlen = batch_tokens.shape[-1]
            batch_ann_labels = {k: torch.LongTensor(batch_labels[k]).to(device)
                                    for k in batch_labels
                                }

            b = batch_size if (idx + batch_size) < num_data else (num_data - idx)
            assert batch_tokens.size() == torch.Size([b, batch_maxlen])
            for vvvvv in batch_ann_labels.values():
                assert vvvvv.size() == torch.Size([b, batch_maxlen])
            assert pad_masks.size() == torch.Size([b, batch_maxlen])

            dataset.append((batch_tokens, batch_ann_labels, pad_masks, batch_doc_ids, batch_sent_offsets, batch_offset_mapping, batch_raw_text))
            idx += batch_size

        print("num_batches=", len(dataset), " | num_data=", num_data)
        return dataset

In [36]:
############# Load dataset #############
train_dataset = MEDataset(train_doc_path, train_label_path, bert_type=bert_type)
valid_dataset = MEDataset(trial_doc_path, trial_label_path, bert_type=bert_type)

print("Dataset created")

############# Create model #############

class OurBERTModel(nn.Module):
    def __init__(self, bert_type="bert-base-cased", tasks=['mp', 'me', 'qual']):
        super(OurBERTModel, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_type)
        self.drop = nn.Dropout(self.bert.config.hidden_dropout_prob)
        self.me_classifier = nn.Linear(self.bert.config.hidden_size, 2)
        if "me" in tasks:
            self.non_lin = nn.Softmax(2)
            self.mp_classifier = nn.Linear(self.bert.config.hidden_size+3, 2)
            self.qual_classifier = nn.Linear(self.bert.config.hidden_size+3, 2)
        else:
            self.mp_classifier = nn.Linear(self.bert.config.hidden_size, 2)
            self.qual_classifier = nn.Linear(self.bert.config.hidden_size, 2)
        

    def forward(self, text, att_mask, token_type=None):
        b, num_tokens = text.shape
        if token_type == None:
            token_type = torch.zeros((b, num_tokens), dtype=torch.long).to(device)
        
        if "roberta" in bert_type:
            outputs = self.bert(text, attention_mask=att_mask)
        else:
            outputs = self.bert(text, attention_mask=att_mask, token_type_ids=token_type)
        classifier_in = self.drop(outputs['last_hidden_state'])

        tags = {}
        if 'me' in tasks:
            tags['me'] = self.me_classifier(classifier_in)
            softmaxed = self.non_lin(tags['me'])
            # Extract features from logits of me and concatenate
            mean_feats = (att_mask.unsqueeze(-1) * softmaxed).sum(1)/att_mask.sum(1).unsqueeze(-1)
            max_feats = (softmaxed[:, :, 1].max(1)[0]).unsqueeze(-1)
            extra_feats = torch.cat([mean_feats, max_feats], -1).unsqueeze(1).expand(-1, num_tokens, -1)
            
            classifier_in = torch.cat([classifier_in, extra_feats], -1)

        if "mp" in tasks:
            tags['mp'] = self.mp_classifier(classifier_in)
        if "qual" in tasks:
            tags['qual'] = self.qual_classifier(classifier_in)

        return tags

model = OurBERTModel(bert_type=bert_type, tasks = ['mp', 'me', 'qual'])
print("Model created")
os.system("nvidia-smi")

model.bert.resize_token_embeddings(len(train_dataset.bert_tok))
# print("Embeddings shape:", model.bert.embeddings.word_embeddings.weight.data.size())
# embedding_size = model.bert.embeddings.word_embeddings.weight.size(1)
# new_embeddings = torch.FloatTensor(2, embedding_size).uniform_(-0.1, 0.1)
# print("new_embeddings shape:", new_embeddings.size())
# new_embedding_weight = torch.cat((model.bert.embeddings.word_embeddings.weight.data,new_embeddings), 0)
# model.bert.embeddings.word_embeddings.weight.data = new_embedding_weight
# print("Updated Embeddings shape:", model.bert.embeddings.word_embeddings.weight.data.size())
# Update model config vocab size
model.bert.config.vocab_size = model.bert.config.vocab_size + 2

print(sum(p.numel() for p in model.parameters()))
model = model.to(device)
print("Detected", torch.cuda.device_count(), "GPUs!")
# model = torch.nn.DataParallel(model)

########## Optimizer & Loss ###########

criterion = torch.nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

########## Training loop ###########

for epoch in range(n_epochs):
    print("\n\n========= Beginning", epoch+1, "epoch ==========")

    train_loss = train(model, train_dataset, criterion, epoch)
    # print("\n====EVALUATING On Training set :====\n")
    # _ = evaluate(model, train_dataset, criterion, epoch)

    print("\n====EVALUATING On Validation set :====\n")
    valid_loss = evaluate(model, tasks, valid_dataset, criterion, epoch)

    epoch_len = len(str(n_epochs))
    print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}]     ' +
                    f'train_loss: {train_loss:.5f} ' +
                    f'valid_loss: {valid_loss:.5f}')
    print("\n", print_msg, "\n")


## Predict on Test set

def predict_spans(dataset, save_path, tasks):
    model.eval()
    predicts = {}

    with torch.no_grad():
        all_tasks = []
        if "me" in tasks: 
            all_tasks.append('me')
        if "mp" in tasks: 
            all_tasks.append('mp')
        if "qual" in tasks: 
            all_tasks.append('qual')
        
        for task_type in all_tasks:
            this_labels_added = []
            for batch in dataset.batched_dataset:
                (texts, token_type_ids, att_masks, token_offsets, quant_data) = batch
                preds = model(texts, att_masks)

                
                preds = preds[task_type]
                for sent_preds, sent_att_masks, sent_token_offsets, sent_quant_data in zip(preds, att_masks, token_offsets, quant_data):

                    this_sentence_positives = []
                    curr_positive_idx = -1
                    for i, (token_preds, token_masks) in enumerate(zip(sent_preds, sent_att_masks)):
                        if token_masks.item() != 0:
                            if token_preds.argmax().item() == 1:
                                if curr_positive_idx == -1:
                                    curr_positive_idx = i
                            else:
                                if curr_positive_idx != -1:
                                    this_sentence_positives.append([curr_positive_idx, i-1])
                                    curr_positive_idx = -1
                                    break
                        else:
                            if curr_positive_idx != -1:
                                this_sentence_positives.append([curr_positive_idx, i-1])
                                curr_positive_idx = -1
                                break

                    # Here convert indices to offsets
                    # assert len(this_sentence_positives) == 1
                    if len(this_sentence_positives) != 0:
                        span_offsets = this_sentence_positives[0]
                        this_label_offs = (sent_token_offsets[span_offsets[0]][0], sent_token_offsets[span_offsets[1]][1])
                        this_labels_added.append([sent_quant_data, (task_type, this_label_offs )])

            json.dump(this_labels_added, open(save_path + '_' + task_type + ".json", 'w+'))

233 15 248
Fixed sentence splitting: {'figs.': 4, 'fig.': 88, 'et al.': 29, 'ref.': 3, 'eq.': 3, 'e.g.': 8, 'i.e.': 3, 'nos.': 2, 'no.': 3, 'spp.': 2}
Loaded and processed data
Loaded tokenizer
LEN VOCAB: 50267
Flattened 233 docs into 855 data points 
Some examples: [['S0006322312001096-1136.tsv', 'Data were drawn from the Whitehall II study with baseline examination in 0000; follow-up screenings in 0000, 0000, and 0000; and additional disease ascertainment from hospital data and registry linkage on 0000 participants (mean age 00.0 years, 00% women) without depressive symptoms at baseline.', 0, [0, 0, [73, 77, 'QUANT', '1', None, [(49, 69, 'ME', '1', {'HasQuantity': 'T1-1'})]]]], ['S0006322312001096-1136.tsv', 'Data were drawn from the Whitehall II study with baseline examination in 0000; follow-up screenings in 0000, 0000, and 0000; and additional disease ascertainment from hospital data and registry linkage on 0000 participants (mean age 00.0 years, 00% women) without depressive symp

Train loss at 0: 0.4228288233280182

====EVALUATING On Validation set :====

Loss Scale: {'me': 1.0, 'mp': 1.0, 'qual': 1.0}

Task: me
[[12142    21]
 [  765    16]]
NOT_ME F1-score: 0.9686477861986437
ME F1-score: 0.039119804400977995
Accu: 0.9392768850432633
F1-Weighted 0.9125630091757778
F1-Avg 0.5038837952998109

Task: mp
[[12563     0]
 [  381     0]]
NOT_MP F1-score: 0.9850629239032422
MP F1-score: 0.0
Accu: 0.9705655129789864
F1-Weighted 0.9560681020547305
F1-Avg 0.4925314619516211

Task: qual
[[12623     0]
 [  321     0]]
NOT_QUAL F1-score: 0.9874447530019166
QUAL F1-score: 0.0
Accu: 0.9752008652657602
F1-Weighted 0.962956977529604
F1-Avg 0.4937223765009583

Validation loss 0.3926658034324646

 [ 7/10]     train_loss: 0.42228 valid_loss: 0.39267 



========= Beginning 9 epoch ==========
Loss Scale: {'me': 1.0, 'mp': 1.0, 'qual': 1.0}
Train loss at 0: 0.39065706729888916

====EVALUATING On Validation set :====

Loss Scale: {'me': 1.0, 'mp': 1.0, 'qual': 1.0}

Task: me
[[12016 

In [41]:
folder_name = output_folder + bert_type.replace('/', '_') + "_final_preds_" + datetime.today().strftime('%Y-%m-%d_%H_%M_%S')
if os.path.isdir(folder_name):
    os.system("rm -rf " + folder_name)
os.mkdir(folder_name)

predict_train_dataset = METestDataset(train_doc_path, test_label_folder + "/train_labels.json")
predict_spans(predict_train_dataset, folder_name + "/train_spans")

predict_trial_dataset = METestDataset(trial_doc_path, test_label_folder + "/trial_labels.json")
predict_spans(predict_trial_dataset, folder_name + "/trial_spans")

predict_test_dataset = METestDataset(eval_doc_path, test_label_folder + "/test_labels.json")
predict_spans(predict_test_dataset, folder_name + "/test_spans")

# don't really save this off...
# torch.save(model.state_dict(), folder_name+"/model.pt")

248 248
Loaded and processed data
Loaded tokenizer
Flattened doc - 846 
Some examples: [['Data were drawn from the Whitehall II study with baseline examination in 0000; follow-up screenings in 0000, 0000, and 0000; and additional disease ascertainment from hospital data and registry linkage on 0000 participants (mean age 00.0 years, 00% women) without depressive symptoms at baseline.', ['S0006322312001096-1136', 1, 'Quantity', 73, 78, 1, '1991;', "{'mods': ['IsCount']}", 0, 296], [73, 78]], ['Data were drawn from the Whitehall II study with baseline examination in 0000; follow-up screenings in 0000, 0000, and 0000; and additional disease ascertainment from hospital data and registry linkage on 0000 participants (mean age 00.0 years, 00% women) without depressive symptoms at baseline.', ['S0006322312001096-1136', 2, 'Quantity', 103, 114, 2, '1997, 2003,', "{'mods': ['IsCount']}", 0, 296], [103, 114]]]
CLS, SEP, PAD, <E>, </E> tokens are: 0 2 1 50265 50266


IndexError: list index out of range